In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from neural_network import NeuralNetworkModel, ChessDataset, ChessEvaluationNeuralNetwork

In [8]:
def eval_to_output(eval: str):
    if eval.startswith('#+'):
        return 10000.0
    elif eval.startswith('#-'):
        return -10000.0
    else:
        return float(eval)

In [9]:
df = pd.read_csv("../../data/chess_data.csv")

X = df['FEN'].apply(NeuralNetworkModel.fen_to_feature_array).tolist()
X = np.array(X, dtype=np.float32)
y = df['Evaluation'].apply(eval_to_output).astype(np.float32).values.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


train_dataset = ChessDataset(X_train, y_train)
test_dataset = ChessDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [10]:
model = ChessEvaluationNeuralNetwork(input_size=X.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(20):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader):.4f}")

Epoch 1, Loss: 933920.7576
Epoch 2, Loss: 869039.5212
Epoch 3, Loss: 817786.7829
Epoch 4, Loss: 772132.5863
Epoch 5, Loss: 746711.7809
Epoch 6, Loss: 720171.1819
Epoch 7, Loss: 699161.9400
Epoch 8, Loss: 680840.6228
Epoch 9, Loss: 665805.2643
Epoch 10, Loss: 647935.0419
Epoch 11, Loss: 635304.0099
Epoch 12, Loss: 613199.6442
Epoch 13, Loss: 594735.2897
Epoch 14, Loss: 581392.1526
Epoch 15, Loss: 557383.3604
Epoch 16, Loss: 536238.8561
Epoch 17, Loss: 512286.3800
Epoch 18, Loss: 503319.3020
Epoch 19, Loss: 491407.9439
Epoch 20, Loss: 471526.6525


In [11]:
model.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()

avg_test_loss = test_loss / len(test_loader)
rmse_error = np.sqrt(avg_test_loss)
print(f"\nTest RMSE: {rmse_error:.4f}")


Test RMSE: 776.5947


In [12]:
torch.save(model.state_dict(), 'neural_network.pth')